# 1. Goal

- 카메라의한 distortion종류
- 카메라의 intrinsic, extrinsic properties 찾기
- distortion을 카메라 properties이용해서 복원하기

# 2. Basics

1. 핀홀 카메라의 두가지 distortions

**(1) Tangentia**:</br>
- 렌즈가 이미지 평면과 완벽히 평행아닐 때</br>
- 한쪽이 예상보다 가깝게 느껴지게함.</br>
$x_{distorted} = [2p_1xy         + p_2(r^2+2x^2)] + x$</br>
$y_{distorted} = [2p_1(r^2+2y^2) + p_2xy] + y$</br>
    
**(2) Radial** : </br>
- 직선을 곡선으로 보이게함.</br>
- 카메라 중심에서 멀수록 심해짐.</br>
$x_{distorted} = (1+k_1r^2+k_2r^4+k_3r^6) x$</br>
$y_{distorted} = (1+k_1r^2+k_2r^4+k_3r^6) y$

<span style ="color:red">
$\rightarrow$ 총 5개의 파라미터를 찾아야함 : Distortion Coeff. = {$p_1,p_2,k_1,k_2,k_3$}
</span>
    
**(3) Intrinsic** : </br>
- 카메라 자체의 특징으로  focal length ($f_x, f_y$)와 optical 센터 ($c_x, c_y$)를 포함한다.</br>
- focal length, optical center는 다음의 3x3 카메라 매트릭스를 구성한다.</br>
- 카메라 매트릭스는 같은 카메라로 찍은 이미지들에 공통으로 쓰일 수 있다.</br>
            
$camera\; matrix = \begin{bmatrix}
f_x & 0 & c_x\\
0 & f_y & c_y\\
0 & 0 & 1
\end{bmatrix}
$</br>
       
       
**(4) Extrinsic** : </br>
- 회전, 선형이동 등의 3D 좌표변환에 해당

# 3. Method

- 보정하는 방법: 알고있는 패턴(체스보드)의 샘플이미지 10개이상 준비, 알고있는 상대적 위치등을 이용하여 파라미터들을 찾는다.
- opencv에서 left01.jpg~left14.jpg의 체스보드 이미지들로 파라미터 찾고, 보정하는 방법을 알아본다.

- (1) 중요한 입력데이터 : 3D위치(object points), 2D위치(image points) (두 검은 사각형이 만나는 위치의)
- (2) 간단히 하기위해서 : 체스보드가 xy평면에 놓여있다라고 즉 Z=0이라고 가정하면 입력데이터는 2D위치로 간단화 된다.
- (3) 스케일 : 체스보드 한 칸의 길이로 스케일링한다. 현 이미지파일속 체스보드 크기 모르니까 1로 normalize함.

# 4. Setup

- 체스판은 보통 8x8 squares, 7x7 internal corners로 이뤄져 있는데. 
- 여기서는 7x6 grid를 사용한다. (그림)
<img src="opencv_lens/opencv_lens_ex_grid.png" width=300 >
-(1) cv.findChessboardCorners() : </br>
    - 함수에 grid정보를 입력하면 이미지에서 코너포인트와 패턴인식 유무를 반환한다. </br>
    - 코너위치 순서는 LEFT$\rightarrow$RIGHT, TOP$\rightarrow$BOTTOM 을 따른다.
-(2) 주의할 점 : </br>
    - 모든 이미지에서 패턴이 찾아지는 것이 아니므로 패턴이 인식되는 이미지들로만 파라미터 찾는데 사용해야 한다.
    - cv.findCircleGrid()로 체스판이 아닌 원형그리드의 이미지로도 가능하다.
-(3) 코너를 찾은 다음? :</br>
    - cv.cornerSubPix() : 정확도를 높이기 위해 사용
    - cv.drawChessboardCorners() : 찾은 패턴 겹쳐그리기?


In [1]:
import numpy as np
import cv2 
import glob
print(cv2.TERM_CRITERIA_EPS)
print(cv2.TERM_CRITERIA_MAX_ITER)
ngridx = 6
ngridy = 7
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
print(criteria)
pt_obj = np.zeros([ngridy*ngridx,3],np.float32)
#pt_obj[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
idx=0
for i in range (ngridx):
    for j in range (ngridy):
        pt_obj[idx]=np.array([j,i,0])
        idx+=1
#pt_obj

2
1
(3, 30, 0.001)


In [2]:
#arrays to store object points and img points from all the images.
pts_obj = []
pts_img = []
pts_obj2 = []
pts_img2 = []

images = glob.glob("./opencv_lens/left*jpg")
for i in range(len(images)):
    image = images[i]
    img_org  = cv2.imread(image)
    img_org2 = cv2.imread(image)
    img = cv2.cvtColor(img_org,cv2.COLOR_BGR2GRAY)
#-------------------------------------------------------- (1) find corners
    ret, corners = cv2.findChessboardCorners(img,(ngridy,ngridx),None)
#-------------------------------------------------------- (2) disregard images with no corners found
    if ret is False:
        print(f"No corners found : {image}")
    else:
        pts_obj.append(pt_obj)
        pts_obj2.append(pt_obj)
        
        pts_img.append(corners)
        cv2.drawChessboardCorners(img_org,(ngridy,ngridx),corners,ret)
        
#-------------------------------------------------------- (3) find more accurate corners
        corners2 = cv2.cornerSubPix(img,corners,(11,11),(-1,-1),criteria)
        pts_img2.append(corners2)
        cv2.drawChessboardCorners(img_org2,(ngridy,ngridx),corners2,ret)
          
        cv2.imshow('img',img_org)
        cv2.imwrite(f"./out_subPix/woSub_out{i:02}.jpg",img_org)
        cv2.imwrite(f"./out_subPix/wSub_out{i:02}.jpg",img_org2)

        cv2.waitKey(1000)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to target thread (0x563e9d4f3cb0)

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to target thread (0x563e9d4f3cb0)

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to target thread (0x563e9d4f3cb0)

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to target thread (0x563e9d4f3cb0)

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to target thread (0x563e9d4f3cb0)

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to target thread (0x563e9d4f3cb0)

QObject::moveToThread: Current thread (0x563e9d4f3cb0) is not the object's thread (0x563e9d841050).
Cannot move to tar

No corners found : ./opencv_lens/left09.jpg
No corners found : ./opencv_lens/left11.jpg


# result

without cv2.cornerSubPix() | with cv2.cornerSubPix
-|-
<img src="./out_subPix/woSub_out00.jpg" width="500">|<img src="./out_subPix/wSub_out00.jpg" width="500">

<span style = "color:red">
# cornerSubPix() is more accurate!!

# 4. Calibration- method1

- corners를 찾아서 pt_obj, pt_img를 모두 찾았다.
- ready to jump into calibration!</br>

(1) find parameters : **cv2.calibrateCamera()** : return</br> 
$^{1)}$camera matrix, </br>
$^{2)}$ distortion coeff., </br>
$^{3)}$ rotation&translation vectors</br>

(2) refine the camera matrix : **cv2.getOptimalNewCameraMatrix()** : return  </br>
$^{1)}$ NEW camera matrix, </br>
$^{2)}$ ROI (helps to crop the image without blank or black fillings.</br>
<span style = "color:blue">
** if scaling parameter (alpha) = 0 : may remove some pixels</br>
** if alpha=1 : it retains all pixels wiht some extra BLACK images.</br>
</span>
(3) undistort the image : **cv2.undistort()**

In [3]:
#-------------------------------------------(1) find parameters
ret, camera_matrix, distortion_coeff, vec_rot, vec_trans = cv2.calibrateCamera(pts_obj2, pts_img2, img.shape[::-1],None,None)
#-------------------------------------------(2) refine the matrix : scaling parameter = 1
test_img = cv2.imread("./opencv_lens/left12.jpg")
h,w = test_img.shape[:2]
NEW_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, distortion_coeff, (w,h),1,(w,h))
#-------------------------------------------(3) undistort the image
out = cv2.undistort(test_img, camera_matrix, distortion_coeff, None, NEW_camera_matrix)
x,y,w,h = roi
out = out[y:y+h,x:x+w]
cv2.imwrite('./out_calibration/calibrated_undistorted_left12.png',out)

True

original | undistorted
-|-
    <img src="./opencv_lens/left12.jpg" height="300">|<img src="./calibrated_undistorted_left12.png" height="300">

# 4. Calibration- method2

(1) find a mapping function : distorted image (original) $\rightarrow$ undistorted image

(2) remap 

In [4]:
#------------------------------------------(1) find a mapping function
xmap,ymap = cv2.initUndistortRectifyMap(camera_matrix, distortion_coeff, None, NEW_camera_matrix, (w,h),5)
#------------------------------------------(2) remap
out2 = cv2.remap(test_img, xmap,ymap,cv2.INTER_LINEAR)

#crop image
x,y,w,h = roi
out2 = out2[y:y+h,x:x+w]
cv2.imwrite("out_calibration/calibrated_undistorted_left12_MAPPING.png",out2)

True

original|after mapping remapping
-|-
<img src="./opencv_lens/left12.jpg" height="300"> | <img src="out_left12_MAPPING.png" height="300">

In [5]:
a = cv2.imread("./out_calibration/calibrated_undistorted_left12_MAPPING.png")
b = cv2.imread("./out_calibration/calibrated_undistorted_left12.png")
print(b.shape)
print(a.shape)

(437, 614, 3)
(415, 603, 3)


# 5. Reprojection Error

- Reprojection 에러? 찾은 파라미터가 얼마나 정확한지 알아보는 방법임.
- Reprojection error=0 가까울수록 더 적확하다.

(1) 카메라 파라미터, distorion 계수, 회전/병진벡터/행렬이 주어졌을 때, 3D-pt_obj 를 2D-pt_img로 바꾸어야 한다. : **cv2.projectPoints()**

(2) $||norm|| = ||transformation - corner finding||$

(3) take mean


In [11]:
mean_error = 0
for i in range(len(pts_obj)):
#---------------------------------------------------------(1) 3D to 2D
    pts_img_projected, _ = cv2.projectPoints(pts_obj[i],vec_rot[i],vec_trans[i],camera_matrix,distortion_coeff)
#---------------------------------------------------------(2) absolute norm : 2D from previous, 2D RE projected from 3D
    error = cv2.norm(pts_img2[i],pts_img_projected,cv2.NORM_L2)/len(pts_img_projected)
    mean_error +=error
#---------------------------------------------------------(3) mean
print(f"tot error : {mean_error/len(pts_obj)}")

tot error : 0.023686000375385676


# 6. Save parameters as a numpy array

In [12]:
results = {}
results["NEW_camera_matrix"]=NEW_camera_matrix
results["camera_matrix"]=camera_matrix
results["distortion_coeff"] = distortion_coeff
results["vec_rot"] = vec_rot
results["vec_trans"] = vec_trans

In [13]:
results

{'NEW_camera_matrix': array([[457.92434692,   0.        , 342.55548195],
        [  0.        , 456.2421875 , 233.34661351],
        [  0.        ,   0.        ,   1.        ]]),
 'camera_matrix': array([[534.07088364,   0.        , 341.53407552],
        [  0.        , 534.11914595, 232.9456526 ],
        [  0.        ,   0.        ,   1.        ]]),
 'distortion_coeff': array([[-2.92971637e-01,  1.07706962e-01,  1.31038377e-03,
         -3.11018812e-05,  4.34798104e-02]]),
 'vec_rot': (array([[-0.45993978],
         [-0.3142018 ],
         [-1.76122223]]),
  array([[-0.35367631],
         [-0.24363035],
         [-1.56874295]]),
  array([[-0.45883216],
         [-0.08848877],
         [-1.33510786]]),
  array([[-0.37843358],
         [-0.18064237],
         [-3.11615996]]),
  array([[-0.32034625],
         [ 0.1597993 ],
         [-1.24149595]]),
  array([[ 0.41531697],
         [ 0.65664497],
         [-1.3373494 ]]),
  array([[-0.29979221],
         [ 0.39216377],
         [-1.4348